In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 모델 파일 경로 설정
model_path = '/content/drive/MyDrive/wildfire_smoke_detector/fire_smoke_model.pth'

Mounted at /content/drive


In [2]:
from google.colab import files
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

# 모델 로드 (학습된 모델 로드)
import torch
from torchvision import models

# 모델 로드 (ResNet18)
model = models.resnet18(pretrained=True)

# 마지막 FC layer 수정 (3개의 클래스: fire, non_fire, smoke)
model.fc = torch.nn.Linear(model.fc.in_features, 3)

# 모델을 GPU로 보내기 (CUDA 사용 여부 체크)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 학습된 모델 로드
model.load_state_dict(torch.load('/content/drive/MyDrive/fire_smoke_model_folder/fire_smoke_model.pth'))  # 모델 경로 확인
model.eval()

# 이미지 업로드
uploaded = files.upload()

# 업로드된 이미지 파일명 가져오기
image_path = list(uploaded.keys())[0]

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

# 이미지 로드 및 변환
image = Image.open(image_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0).to(device)

# 모델 예측
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)

# 클래스 이름 (예: 'fire', 'non_fire', 'smoke')
class_names = ['Smoke', 'fire', 'non_fire']  # 정확한 클래스 순서 확인 필요
print(f"🔥 예측 결과: {class_names[predicted.item()]}")

# 이미지를 출력 (선택사항)
plt.imshow(image)
plt.title(f"예측 결과: {class_names[predicted.item()]}")
plt.show()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.